In [ ]:
import sklearn
print(sklearn.__version__)

1.6.1


In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import joblib
import os

# --- Configuration ---
# Set page configuration for better aesthetics
st.set_page_config(
    page_title="Employee Salary Predictor",
    page_icon="💰",
    layout="centered",
    initial_sidebar_state="auto"
)

# --- Load Model and Preprocessors ---
@st.cache_resource # Use st.cache_resource for loading large objects like models
def load_artifacts():
    try:
        model_pipeline = joblib.load('random_forest_salary_model.joblib')
        label_encoder = joblib.load('label_encoder.joblib')
        preprocessor = joblib.load('preprocessor.joblib')
        model_feature_names = joblib.load('model_feature_names.joblib') # This will be the list of all features used during training
        return model_pipeline, label_encoder, preprocessor, model_feature_names
    except FileNotFoundError as e:
        st.error(f"Error loading model artifacts. Make sure all .joblib files are in the same directory as this script. Missing: {e.filename}")
        return None, None, None, None # Return None for all variables
    except Exception as e:
        st.error(f"An unexpected error occurred while loading artifacts: {e}")
        return None, None, None, None # Return None for all variables

model_pipeline, label_encoder, preprocessor, model_feature_names = load_artifacts()

# --- Stop the app if artifacts are not loaded ---
if not all([model_pipeline, label_encoder, preprocessor, model_feature_names]):
    st.warning("Model artifacts not loaded. Please ensure the required .joblib files are present.")
    st.stop()


# --- Define Default Values for Non-User-Provided Features ---
# These defaults are based on the median/mode of the training data from the Colab notebook.
# In a real-world scenario, you might derive these from a saved statistics file or directly from the preprocessor.
# For simplicity here, we'll hardcode based on typical values or assume 0 for gain/loss.
# It's better to get these from the actual training data or the fitted preprocessor if possible.
# For this example, we'll use values derived from the Colab notebook's X_train.
# You might need to run the Colab notebook and inspect X_train.median() and X_train.mode() for precise values.
default_values = {
    'educational-num': 10.0,  # Median from a typical adult dataset
    'marital-status': 'Married-civ-spouse', # Mode from a typical adult dataset
    'relationship': 'Husband', # Mode from a typical adult dataset
    'race': 'White', # Mode from a typical adult dataset
    'capital-gain': 0.0, # Often 0 for most people
    'capital-loss': 0.0  # Often 0 for most people
}

# Original column names from the dataset (excluding 'income')
original_data_columns = ['age', 'workclass', 'fnlwgt', 'education', 'educational-num',
                         'marital-status', 'occupation', 'relationship', 'race',
                         'gender', 'capital-gain', 'capital-loss', 'hours-per-week',
                         'native-country']

# --- Streamlit App Title and Description ---
st.title("💰 Employee Salary Predictor")
st.markdown("""
Welcome to the Employee Salary Predictor!
Fill in the details below to predict if an employee's income is <=50K or >50K.
""")

# --- Input Form ---
st.header("Employee Details")

with st.form("prediction_form"):
    col1, col2 = st.columns(2)

    with col1:
        age = st.slider("Age", 17, 90, 30)
        workclass = st.selectbox("Workclass", ['Private', 'Self-emp-not-inc', 'Local-gov', 'State-gov',
                                               'Self-emp-inc', 'Federal-gov', 'Without-pay', 'Never-worked'])
        fnlwgt = st.number_input("Final Weight (fnlwgt)", min_value=10000, max_value=1500000, value=200000)
        education = st.selectbox("Education", ['HS-grad', 'Some-college', 'Bachelors', 'Masters', 'Assoc-voc',
                                                '11th', 'Assoc-acdm', '10th', '7th-8th', 'Prof-school',
                                                '9th', '12th', 'Doctorate', '5th-6th', '1st-4th', 'Preschool'])

    with col2:
        occupation = st.selectbox("Occupation", ['Prof-specialty', 'Craft-repair', 'Exec-managerial', 'Adm-clerical',
                                                 'Sales', 'Other-service', 'Machine-op-inspct', 'Transport-moving',
                                                 'Handlers-cleaners', 'Farming-fishing', 'Tech-support',
                                                 'Protective-serv', 'Priv-house-serv', 'Armed-Forces'])
        gender = st.radio("Gender", ['Male', 'Female'])
        hours_per_week = st.slider("Hours per Week", 1, 99, 40)
        native_country = st.selectbox("Native Country", ['United-States', 'Mexico', 'Philippines', 'Germany',
                                                         'Puerto-Rico', 'Canada', 'El-Salvador', 'India',
                                                         'England', 'Cuba', 'Jamaica', 'South', 'China',
                                                         'Italy', 'Dominican-Republic', 'Vietnam', 'Guatemala',
                                                         'Columbia', 'Poland', 'Haiti', 'Portugal', 'Iran',
                                                         'Taiwan', 'Greece', 'Nicaragua', 'Peru', 'Ecuador',
                                                         'France', 'Ireland', 'Hong', 'Thailand', 'Cambodia',
                                                         'Trinadad&Tobago', 'Laos', 'Yugoslavia', 'Outlying-US(Guam-USVI-etc)',
                                                         'Honduras', 'Hungary', 'Scotland', 'Holand-Netherlands'])

    submitted = st.form_submit_button("Predict Salary")

    if submitted:
        # Create a dictionary for the input features, including default values for missing ones
        input_data = {
            'age': age,
            'workclass': workclass,
            'fnlwgt': fnlwgt,
            'education': education,
            'educational-num': default_values['educational-num'], # Default value
            'marital-status': default_values['marital-status'],   # Default value
            'occupation': occupation,
            'relationship': default_values['relationship'],       # Default value
            'race': default_values['race'],                       # Default value
            'gender': gender,
            'capital-gain': default_values['capital-gain'],       # Default value
            'capital-loss': default_values['capital-loss'],       # Default value
            'hours-per-week': hours_per_week,
            'native-country': native_country
        }

        # Convert input data to a DataFrame, ensuring column order matches original training data
        input_df = pd.DataFrame([input_data])
        # Reorder columns to match the original training features (X.columns in Colab)
        input_df = input_df[original_data_columns]

        # Make prediction
        try:
            prediction_encoded = model_pipeline.predict(input_df)
            predicted_income = label_encoder.inverse_transform(prediction_encoded)

            st.success(f"Predicted Income: **{predicted_income[0]}**")
            if predicted_income[0] == '>50K':
                st.balloons()
            st.info("This prediction indicates whether the employee's income is likely to be above or below $50,000.")
        except Exception as e:
            st.error(f"An error occurred during prediction: {e}")
            st.warning("Please ensure the input values are valid and try again.")

st.markdown("---")
st.markdown("Developed by Gemini for your salary prediction needs.")

2025-07-27 12:24:34.189 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-27 12:24:34.199 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-27 12:24:34.201 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-27 12:24:34.203 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-27 12:24:34.203 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-27 12:24:34.210 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-27 12:24:34.212 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-27 12:24:34.213 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator(_form_data=FormData(form_id='prediction_form'))